In [ ]:
#請下載台北市youbike及時資料(json),解析資料!

#請幫我查出目前可借車輛數為3輛以內的站點
#請幫我查出目前可還車輛數為3輛以內的站點
#請將維護的站點移除
'''
請過濾
sna:str(場站中文名稱)
sarea:str(場站區域)
mday:str(資料更新時間)
ar:str(地點)
act:bool(全站禁用狀態)
total:int(場站總停車格)
available_rent_bikes:int(場站目前車輛數量)
available_return_bikes:int(空位數量)
latitude:float(緯度)
longitude:float(經度)
'''
#youbike = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'


In [2]:
import requests
import pyinputplus as pyip
from pprint import pprint
from requests import Response
from pydantic import BaseModel,RootModel,Field,ValidationError,field_validator
from random import choices

class UB(BaseModel):
    場站中文名稱:str = Field(alias='sna')
    場站區域:str = Field(alias='sarea')
    資料更新時間:str = Field(alias='mday')
    地點:str = Field(alias='ar')
    act:bool
    場站總停車格:int = Field(alias='total')
    場站目前車輛數量:int = Field(alias='available_rent_bikes')
    空位數量:int = Field(alias='available_return_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')

class Root(RootModel):
    root:list[UB]

def main():
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res:Response = requests.get(youbike_url)
    all_data = res.json()

    try:
        root:Root = Root.model_validate(all_data)
    except ValidationError as error:
        print(error)

    data_list:list[dict] = root.model_dump()
    filter_list:list[dict] = list(filter(lambda item:False if item['act']==0 else True,data_list))
    filter_list_rent_less3:list[dict] = list(filter(lambda item:True if item['場站目前車輛數量']<3 else False,data_list))
    filter_list_return_less3:list[dict] = list(filter(lambda item:True if item['空位數量']<3 else False,data_list))

    print(f"台北市YouBike有{len(data_list)}站點，目前正常運作站點共{len(filter_list)}個")

    num = pyip.inputInt("請輸入隨機顯示的站點數(1~5): ", min=1, max=5)
    print(num)

    print(f"\n可借車輛數為3輛以內的站點有{len(filter_list_rent_less3)}站")
    num_list:list[dict] = choices(filter_list_rent_less3, k=num)
    for sta in num_list:
            seq = filter_list_rent_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

    print(f"\n可還車輛數為3輛以內的站點有{len(filter_list_return_less3)}站")
    num_list:list[dict] = choices(filter_list_return_less3, k=num)
    for sta in num_list:
            seq = filter_list_return_less3.index(sta)
            print(f"No.{seq}")
            pprint(sta, indent=2, sort_dicts=False)

if __name__ == '__main__':
     main()

台北市YouBike有1420站點，目前正常運作站點共1411個
請輸入隨機顯示的站點數(1~5): 2

可借車輛數為3輛以內的站點有376站
No.27
{ '場站中文名稱': 'YouBike2.0_捷運信義安和站(2號出口)',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 09:32:19',
  '地點': '信義路四段188號前',
  'act': True,
  '場站總停車格': 24,
  '場站目前車輛數量': 2,
  '空位數量': 22,
  '緯度': 25.03304,
  '經度': 121.55171}
No.139
{ '場站中文名稱': 'YouBike2.0_仁愛金山路口',
  '場站區域': '中正區',
  '資料更新時間': '2024-06-13 09:34:15',
  '地點': '仁愛路二段31號前',
  'act': True,
  '場站總停車格': 9,
  '場站目前車輛數量': 0,
  '空位數量': 9,
  '緯度': 25.0384,
  '經度': 121.52801}

可還車輛數為3輛以內的站點有146站
No.80
{ '場站中文名稱': 'YouBike2.0_週美里活動廣場',
  '場站區域': '內湖區',
  '資料更新時間': '2024-06-13 09:29:20',
  '地點': '潭美街139號對側',
  'act': True,
  '場站總停車格': 17,
  '場站目前車輛數量': 15,
  '空位數量': 2,
  '緯度': 25.0539,
  '經度': 121.58084}
No.120
{ '場站中文名稱': 'YouBike2.0_松山家商',
  '場站區域': '信義區',
  '資料更新時間': '2024-06-13 09:21:22',
  '地點': '松山路655號(前)',
  'act': True,
  '場站總停車格': 25,
  '場站目前車輛數量': 23,
  '空位數量': 2,
  '緯度': 25.0347,
  '經度': 121.58018}
